# Construction of a public dataset of Blood Bowl matches played on FUMBBL.com

This blogpost is about **Blood Bowl**, a boardgame I started playing last year. The goal of this blog post is to use Python API and HTML scraping to fetch online Blood Bowl match outcome data, and to create a structured dataset ready for analysis and visualization. This blogpost is written as a Jupyter notebook containing Python code, and is fully reproducible. The idea is to make Blood Bowl data analysis accessible to others. Using open source tooling reduces the barriers for others to build on other people’s work.


In [ ]:
import random
import time
import os

from isoweek import Week

import requests # API library

import numpy as np
import pandas as pd
import json

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Blood Bowl online: FUMBBL 

The **FUMBBL** website (https://fumbbl.com) is one big pile of data. From coach pages, with their teams, to team rosters, with players, and match histories. It's all there.

To obtain **FUMBBL** data, we need to fetch it match by match, team by team. To do so, the site creator Christer Kaivo-oja, from Sweden, has made an API that allows us to easily fetch data. 

# Using Python requests package to fetch data
We use the [Python **Requests** library](https://docs.python-requests.org/en/latest/) to make the API call over HTTPS and obtain the response from the FUMBLL server. The response is in the JSON format, a [light-weight data-interchange format](https://www.json.org/json-en.html) which is both easy to read and write for humans, and easy to parse and generate by computers. So this makes it a natural choice for an API.
The full documentation of the API can be found at (https://fumbbl.com/apidoc/).


# Data structure

We go with a flat data frame with **rows for each match**, and columns for the various variables associated with each match.
These would include:

* Coach ids
* Team races
* Team ids
* Date of the match
* Outcome (Touchdowns of both teams)

With this basic structure, we can add as many match related variables in the future, keeping the basic structure (each row is a match) unchanged.

So lets get the match data!



# Step 1: API scraping the match data: df_matches

So we are mostly interested in the current ruleset, this is `BB2020`. This ruleset became available in **FUMBBL** at september 1st 2021, and two months later, some 5000 games have been played. We also want to compare with the previous ruleset, where we have much more data available. 
The dataset start with match `4216258` played on august 1st, 2020. This covers roughly 12 months of `BB2016` ruleset matches, after that it switches to predominantly `BB2020` matches.

We collect match data by looping over `match_id`. We store the full JSON file on disk, so we avoid repeat API calls in the future.

**VERY IMPORTANT: We do not want to overload the **FUMBBL** server, so we make only three API requests per second. In this way, the server load is hardly affected and it can continue functioning properly for all the Blood Bowl coaches playing their daily games!**

In [ ]:
%run src/write_json_file.py

In [ ]:
def update_match_jsons(full_run, begin_match, end_match, verbose = False):

    n_matches = end_match - begin_match

    print("matches to grab:")
    print(n_matches)

    if(full_run):
        for i in range(n_matches):
            if i % 100 == 0: 
                if verbose:
                    # write progress report
                    print(i, end='')
                    print(".")

            match_id = end_match - i

            dirname = "raw/match_html_files/" + str(match_id)[0:4]
            if not os.path.exists(dirname):
                os.makedirs(dirname)

            fname_string = dirname + "/match_" + str(match_id) + ".json"

            # check if file already exists, else scrape it
            try:
                f = open(fname_string, mode = "rb")

            except OSError as e:
              # scrape it
                api_string = "https://fumbbl.com/api/match/get/" + str(match_id)

                match = requests.get(api_string)
                match = match.json()

                write_json_file(match, fname_string)
                if verbose:
                    print("x", end = '')
                time.sleep(0.3)
            else:
                # file already present
                if verbose:
                    print("o", end = '')
                continue

update_match_jsons(full_run = 0, begin_match = 4460944, end_match = 4474450, verbose=True) # takes 17s if no new matches

## Processing the match JSONS

In [ ]:
def process_match_jsons(full_run, begin_match, end_match, verbose = True, target_file = None):


    n_matches = end_match - begin_match

    print("matches to process:")
    print(n_matches)

    if(full_run):
        target = 'raw/df_matches_' + time.strftime("%Y%m%d_%H%M%S") + '.h5'
        print(target)

        match_id_list = []
        replay_id = []
        tournament_id = []
        division_id = []
        division_name = []
        scheduler = []
        match_date = []
        match_time = []
        match_conceded = []
        team1_id = []
        team2_id = []
        # touchdowns
        team1_score = []
        team2_score = []
        # casualties
        team1_cas_bh = []
        team1_cas_si = []
        team1_cas_rip = []
        team2_cas_bh = []
        team2_cas_si = []
        team2_cas_rip = []
        # other
        team1_roster_id = []
        team2_roster_id = []
        team1_coach_id = []
        team2_coach_id = []
        team1_race_name = []
        team2_race_name = []
        team1_value = []
        team2_value = []

        for i in range(n_matches):
            if i % 10000 == 0: 
                if verbose:
                    # write progress report
                    print(i, end='')
                    print(".", end='')

            match_id = end_match - i

            dirname = "raw/match_html_files/" + str(match_id)[0:4]
            if not os.path.exists(dirname):
                os.makedirs(dirname)

            fname_string = dirname + "/match_" + str(match_id) + ".json"
             
            
            # read compressed json file
            with open(fname_string, mode = "rb") as f:
                match = json.load(f)

            if match: # fix for matches that do not exist
                match_id_list.append(match['id'])
                replay_id.append(match['replayId'])
                tournament_id.append(match['tournamentId']) # key to tournament table
                division_id.append(match['divisionId'])
                division_name.append(match['division'])
                scheduler.append(match['scheduler'])
                match_date.append(match['date'])
                match_time.append(match['time'])
                match_conceded.append(match['conceded'])
                team1_id.append(match['team1']['id'])
                team2_id.append(match['team2']['id'])
                # touchdowns
                team1_score.append(match['team1']['score'])
                team2_score.append(match['team2']['score'] ) 
                # casualties
                team1_cas_bh.append(match['team1']['casualties']['bh'])
                team1_cas_si.append(match['team1']['casualties']['si'])
                team1_cas_rip.append(match['team1']['casualties']['rip'])
                team2_cas_bh.append(match['team2']['casualties']['bh'])
                team2_cas_si.append(match['team2']['casualties']['si'])
                team2_cas_rip.append(match['team2']['casualties']['rip'])
                # other
                team1_roster_id.append(match['team1']['roster']['id'])
                team2_roster_id.append(match['team2']['roster']['id'] )           
                team1_coach_id.append(match['team1']['coach']['id'])
                team2_coach_id.append(match['team2']['coach']['id'])
                team1_race_name.append(match['team1']['roster']['name'] )
                team2_race_name.append(match['team2']['roster']['name'] )
                team1_value.append(match['team1']['teamValue'])
                team2_value.append(match['team2']['teamValue'])
                
            else:
                # skip match
                continue 
                
        data = zip(match_id_list, replay_id, tournament_id, division_id, division_name, scheduler, match_date, match_time,  match_conceded, 
                   team1_id, team1_coach_id, team1_roster_id, team1_race_name, team1_value, team1_cas_bh, team1_cas_si, team1_cas_rip,
                   team2_id, team2_coach_id, team2_roster_id, team2_race_name, team2_value, team2_cas_bh, team2_cas_si, team2_cas_rip,
                   team1_score, team2_score)

        df_matches = pd.DataFrame(data, columns=['match_id', 'replay_id', 'tournament_id', 'division_id', 'division_name', 'scheduler', 'match_date', 'match_time',  'match_conceded',
        'team1_id', 'team1_coach_id', 'team1_roster_id', 'team1_race_name', 'team1_value', 'team1_cas_bh', 'team1_cas_si', 'team1_cas_rip',
        'team2_id', 'team2_coach_id', 'team2_roster_id', 'team2_race_name', 'team2_value', 'team2_cas_bh', 'team2_cas_si', 'team2_cas_rip',
        'team1_score', 'team2_score'])
      
        df_matches = df_matches.sort_values(by=['match_id']).reset_index(drop=True)
        df_matches.to_hdf(target, key='df_matches', mode='w')
    else:
        # read from hdf5 file    
        df_matches = pd.read_hdf(target_file) 

    return df_matches


#df_matches = process_match_jsons(full_run = 1, begin_match = 4460944, end_match = 4474450, verbose= True)
#df_matches = process_match_jsons(full_run = 0, begin_match = 4460944, end_match = 4474450, verbose= True, target_file = 'raw/df_matches_20230728_082435.h5')
df_matches = process_match_jsons(full_run = 0, begin_match = 4216259, end_match = 4474450, verbose= True, target_file = 'raw/df_matches_20230728_215320.h5')
#df_matches = process_match_jsons(full_run = 1, begin_match = 4216259, end_match = 4474450, verbose= True)

# here we need as begin_match the very first match on disk

df_matches.shape

In [ ]:
258191-244646

## Data prep: fixing the datatypes, creating derived variables

Since we manually filled the `pandas` DataFrame, most of the columns are now of `object` datatype.
We need to change this to be able to work properly with the data, as well as store it properly.
Here I convert each column manually, however I later found out about `DataFrame.infer_objects()`, that can detect the proper dtype automatically.
This I will try next time.

In [ ]:
# pm convert into function

# convert object dtype columns to proper pandas dtypes datetime and numeric
df_matches['match_date'] = pd.to_datetime(df_matches.match_date) # Datetime object

# calculate match score difference
df_matches['team1_win'] = np.sign(df_matches['team1_score'] - df_matches['team2_score'])
df_matches['team2_win'] = np.sign(df_matches['team2_score'] - df_matches['team1_score'])

# mirror match
df_matches['mirror_match'] = 0
df_matches.loc[df_matches['team1_race_name'] == df_matches['team2_race_name'], 'mirror_match'] = 1

# add total CAS
df_matches['team1_cas'] = df_matches['team1_cas_bh'] + df_matches['team1_cas_si'] + df_matches['team1_cas_rip']

# add total CAS
df_matches['team2_cas'] = df_matches['team2_cas_bh'] + df_matches['team2_cas_si'] + df_matches['team2_cas_rip']

# mirror matches
df_matches.query('mirror_match == 1').shape

In [ ]:
df_matches.info()

## Dataprep: transforming the team values

In Blood Bowl, teams can develop themselves over the course of multiple matches. The winnings of each match can be spend on buying new, stronger players, or replace the players that ended up getting injured or even killed. In addition, players receive so-called *star points* for important events, such as scoring, or inflicting a casualty on the opponent. Therefore, a balancing mechanism is needed when a newly created "rookie" team is facing a highly developed opposing team with lots of extra skills and strong players. 

Blood Bowl solves this by calculating for both teams their **Current team value**.
The **Team value difference** for a match determines the amount of gold that the weaker team can use to buy so-called **inducements**.
These inducements are temporary, and can consists of a famous "star player" who joins the team just for this match. Another popular option is to hire a wizard that can be used to turn one of the opposing players into a frog.

It is well known that the win rates of the teams depend on how developed a team is. For example, Amazons are thought to be strongest at low team value, as they already start out with lots of *block* and *dodge* skills, whereas a Chaos team start out with almost no skills.
So if we compare win rates, we would like take into account the current team value. 
Now as this can differ between the two teams in a match up, I reasoned that the highest team value is most informative about the average strength level of both teams, because of the inducement mechanism described above. (In the next step, we will add information on inducements)

In the dataset, we have for each match the current team values of both teams as a text string. 
We transform the text string `1100k` into an integer number `1100`, so that we can calculated the difference as `tv_diff`, and pick for each match the maximum team value and store it as `tv_match`. Finally, we create a team value bin `tv_bin` to be able to compare win rates for binned groups of matches where races have comparable team strength / team development.


In [ ]:
#PM make function transform_data
def transform_data():
    # do nothing
    print("")

# convert team value 1100k to 1100 integer and and above / below median (= low / high TV)
df_matches['team1_value'] = df_matches['team1_value'].str.replace('k$', '')
df_matches['team1_value'] = df_matches['team1_value'].fillna(0).astype(np.int64)

df_matches['team2_value'] = df_matches['team2_value'].str.replace('k$', '')
df_matches['team2_value'] = df_matches['team2_value'].fillna(0).astype(np.int64)

df_matches['tv_diff'] = np.abs(df_matches['team2_value'] - df_matches['team1_value'])
df_matches['tv_diff2'] = df_matches['team2_value'] - df_matches['team1_value']

df_matches['tv_match'] = df_matches[["team1_value", "team2_value"]].max(axis=1)

df_matches['tv_bin'] = pd.cut(df_matches['tv_match'], 
    bins = [0, 950, 1250,1550, 1850, float("inf")], 
    labels=['< 950K', '1.1M', '1.4M', '1.7M', '> 1850K']
)

df_matches['tv_bin2'] = pd.cut(df_matches['tv_match'], 
    bins = [0, 950, 1050, 1150, 1250, 1350, 1450, 1550, float("inf")], 
    labels=['< 950K', '1.0M', '1.1M', '1.2M',  '1.3M', '1.4M', '1.5M', '> 1550K']
)


## Dropping empty matches

Some match_id's do not have match information attached to them, presumably these matches were not played or some real life event interfered. These match_ids are dropped from the dataset to get rid of the NAs in all the columns.

In [ ]:
len(df_matches)
df_matches = df_matches.dropna(subset=['match_date'])

In [ ]:
df_matches.shape

## Dataprep: getting the dates right

To see time trends, its useful to aggregate the data by week. For this we add `week_number` for each date, and from this week number, we convert back to a date to get a `week_date`. This last part is useful for plotting with `plotnine`, as this treats dates in a special way
We use the ISO definition of week, this has some unexpected behavior near the beginning / end of each year. 

The data starts in week 36 (september) of 2020, and stops halfway march 2022.


In [ ]:
df_matches['week_number'] = df_matches['match_date'].dt.isocalendar().week

# cannot serialize numpy int OR Int64 when writing HDF5 file, so we go for plain int as all NAs are gone now
df_matches['week_number'] = df_matches['week_number'].fillna(0).astype(int)

# add year based on match ISO week
df_matches['year'] = df_matches['match_date'].dt.isocalendar().year.astype(int)

df_matches['week_year'] = df_matches['year'].astype(str) + '-' + df_matches['week_number'].astype(str)

# use a lambda function since isoweek.Week is not vectorized 
df_matches['week_date'] = pd.to_datetime(df_matches.apply(lambda row : Week(int(row["year"]),int(row["week_number"])).monday(),axis=1))



# Step 2: HTML Scraping more match related data

Next, we collect more match related data to add to  `df_matches`.
For example, the **inducements** and **coach rankings**, as well as match performance stats such as total passing distance `pass`. 
This information is not available through the API, but each played match has an associated HTML page at https://fumbbl.com/FUMBBL.php?page=match with more info.

Since we have to fetch the complete HTML page for each match anyways, I decided to split the proces in two steps:

In the first step, the HTML pages for the desired matches are fetched and stored on disk. 
After some optimization fetching 1K matches takes 10 min. So 6K matches per hour.

(Total file size for 154K files is 7GB. These files cannot be stored on Github.)

In the second step, the HTML pages are processed with `BeautifulSoup` to extract inducments and coach rankings.

In [ ]:
# https://thehftguy.com/2020/07/28/making-beautifulsoup-parsing-10-times-faster/

import lxml
import cchardet
from bs4 import BeautifulSoup
import re
import gzip

def do_html_scraping(full_run, begin_match, end_match, verbose = False):

    print("matches to scrape: ")
    n_matches = end_match - begin_match

    print(n_matches)

    if(full_run):
        for i in range(n_matches):
            match_id = end_match - i
            api_string = "https://fumbbl.com/FUMBBL.php?page=match&id=" + str(match_id)

            response = requests.get(api_string)

            dirname = "raw/match_html_files/" + str(match_id)[0:4]
            if not os.path.exists(dirname):
                os.makedirs(dirname)

            fname_string = dirname + "/match_" + str(match_id) + ".html.gz"
            
            with gzip.open(fname_string, mode = "wb") as f:
                f.write(response.text.encode("utf-8"))
                f.close()

            
            if i % 1000 == 0: 
                if verbose:
                # write progress report
                    print(i, end='')
                    print(".", end='')


do_html_scraping(full_run = 0, begin_match = 4460944, end_match = 4474450, verbose= True)  

# Processing the match HTML files

I highly recommend [this tutorial](https://hackersandslackers.com/scraping-urls-with-beautifulsoup/) for a great introduction to `BeautifulSoup`.
It allows easy access to the information contained within HTML "div" tags that partition the web page in different sections. 

In addition, to clean up the scraped text, I used the **re** Python module (Regular expressions), part of the [Python standard library](https://docs.python.org/3/library/index.html) to extract the actual inducements from the text string that contains them.

Now it takes 1.5 min for 1K matches. 40K matches, expect 1h. check


In [ ]:
def do_all_the_stuff(full_run, begin_match = None, end_match = None, verbose = False):

    if(full_run):
        target = 'raw/df_matches_html_' + time.strftime("%Y%m%d_%H%M%S") + '.h5'

        print(target)

        n_matches = end_match - begin_match
    
        print(n_matches)
        match_id = [] # used CTRL + D to add to selection
        team1_inducements = []
        team2_inducements = []
        coach1_ranking = []
        coach2_ranking = []
        team1_comp = []
        team2_comp = []
        team1_pass = []
        team2_pass = []
        team1_rush = []
        team2_rush = []
        team1_block = []
        team2_block = []
        team1_foul = []
        team2_foul = []

        for i in range(n_matches):
            match_id_tmp = end_match - i
            dirname = "raw/match_html_files/" + str(match_id_tmp)[0:4]

            # PM first check gz, if not exist then check for unzipped html
            fname_string = dirname + "/match_" + str(match_id_tmp) + ".html"        
            fname_string_gz = dirname + "/match_" + str(match_id_tmp) + ".html.gz"        

            with gzip.open(fname_string_gz, mode = "rb") as f:
                soup = BeautifulSoup(f, 'xml')

            if soup.find("div", {"class": "matchrecord"}) is not None:
                # match record is available
                inducements = soup.find_all("div", class_="inducements")

                pattern = re.compile(r'\s+Inducements: (.*)\n')

                match = re.match(pattern, inducements[0].get_text())
                if match:
                    team1_inducements_tmp = match.group(1)
                else:
                    team1_inducements_tmp = ''

                match = re.match(pattern, inducements[1].get_text())
                if match:
                    team2_inducements_tmp = match.group(1)
                else:
                    team2_inducements_tmp = ''

                coach_info = soup.find_all("div", class_="coach")
                # grab the ranking
                coach1_ranking_tmp = coach_info[0].get_text()
                coach2_ranking_tmp = coach_info[1].get_text()

                # match performance stats
                div = soup.find_all('div', class_= "player foot")
                # passing completions
                regex = re.compile('.*front comp statbox.*')
                team1_comp_tmp = div[0].find("div", {"class" : regex}).get_text()
                team2_comp_tmp = div[1].find("div", {"class" : regex}).get_text()                   
                # passing distance in yards
                regex = re.compile('.*back pass statbox.*')
                team1_pass_tmp = div[0].find("div", {"class" : regex}).get_text()
                team2_pass_tmp = div[1].find("div", {"class" : regex}).get_text()
                # rushes
                regex = re.compile('.*back rush statbox.*')
                team1_rush_tmp = div[0].find("div", {"class" : regex}).get_text()
                team2_rush_tmp = div[1].find("div", {"class" : regex}).get_text()
                # block
                regex = re.compile('.*back block statbox.*')
                team1_block_tmp = div[0].find("div", {"class" : regex}).get_text()
                team2_block_tmp = div[1].find("div", {"class" : regex}).get_text()
                # foul
                regex = re.compile('.*back foul statbox.*')
                team1_foul_tmp = div[0].find("div", {"class" : regex}).get_text()
                team2_foul_tmp = div[1].find("div", {"class" : regex}).get_text()

                match_id.append(match_id_tmp) # append for single item, extend for multiple items
                team1_inducements.append(team1_inducements_tmp)
                team2_inducements.append(team2_inducements_tmp)
                coach1_ranking.append(coach1_ranking_tmp)
                coach2_ranking.append(coach2_ranking_tmp)
                team1_comp.append(team1_comp_tmp)
                team2_comp.append(team2_comp_tmp)
                team1_pass.append(team1_pass_tmp)
                team2_pass.append(team2_pass_tmp)
                team1_rush.append(team1_rush_tmp)
                team2_rush.append(team2_rush_tmp)
                team1_block.append(team1_block_tmp)
                team2_block.append(team2_block_tmp)
                team1_foul.append(team1_foul_tmp)
                team2_foul.append(team2_foul_tmp)
            
            if i % 1000 == 0: 
            # write progress report
                print(i, end='')
                print(".", end='')

                data = zip(match_id, team1_inducements, team2_inducements, 
                                        coach1_ranking, coach2_ranking, team1_comp, team2_comp,
                                        team1_pass, team2_pass, team1_rush, team2_rush,
                                        team1_block, team2_block, team1_foul, team2_foul)

                df_matches_html = pd.DataFrame(data, columns = ['match_id', 'team1_inducements', 'team2_inducements',
                'coach1_ranking', 'coach2_ranking', 'team1_comp', 'team2_comp',
                'team1_pass', 'team2_pass', 'team1_rush', 'team2_rush',
                'team1_block', 'team2_block', 'team1_foul', 'team2_foul'])

                df_matches_html.to_hdf(target, key='df_matches_html', mode='w')

        # write data as hdf5 file
        data = zip(match_id, team1_inducements, team2_inducements, 
                                coach1_ranking, coach2_ranking, team1_comp, team2_comp,
                                team1_pass, team2_pass, team1_rush, team2_rush,
                                team1_block, team2_block, team1_foul, team2_foul)

        df_matches_html = pd.DataFrame(data, columns = ['match_id', 'team1_inducements', 'team2_inducements',
        'coach1_ranking', 'coach2_ranking', 'team1_comp', 'team2_comp',
        'team1_pass', 'team2_pass', 'team1_rush', 'team2_rush',
        'team1_block', 'team2_block', 'team1_foul', 'team2_foul'])

        df_matches_html.to_hdf(target, key='df_matches_html', mode='w')
    else:
        # read from hdf5 file    
        df_matches_html1 = pd.read_hdf('raw/df_matches_html_20220315_220825.h5') 
        df_matches_html2 = pd.read_hdf('raw/df_matches_html_20220316_133752.h5') 
        df_matches_html3 = pd.read_hdf('raw/df_matches_html_20220608_054453.h5')
        df_matches_html4 = pd.read_hdf('raw/df_matches_html_20230115_133734.h5')
        df_matches_html5 = pd.read_hdf('raw/df_matches_html_20230526_080359.h5')
        df_matches_html6 = pd.read_hdf('raw/df_matches_html_20230728_185951.h5')
        df_matches_html = pd.concat([df_matches_html1, df_matches_html2, 
                                     df_matches_html3, df_matches_html4, 
                                     df_matches_html5, df_matches_html6], ignore_index= True)

    return df_matches_html

#df_matches_html = do_all_the_stuff(full_run = 1, begin_match = 4460944, end_match = 4474450, verbose= True)
df_matches_html = do_all_the_stuff(full_run = 0, verbose= True)

df_matches_html.info()

In [ ]:
#%load_ext line_profiler
#%lprun -f do_all_the_stuff do_all_the_stuff()
# df_matches_html_20230115_133734


Conclusion of the profiling: >95% of the time is spend within Beautiful Soup. Switch to xml parser doubled processing speed.
No further room for improvements.

## Fix the performance stats

The completions have a few weird values like `4/1`, we drop the slash and the value behind that.
`-` is converted to 0 when the string ends directly after the `-` character, i.e. `-` becomes 0 but `-1` becomes -1.

In [ ]:
df_matches_html['team1_comp'] = df_matches_html['team1_comp'].str.replace(r'(/).*','')
df_matches_html['team1_comp'] = pd.to_numeric(df_matches_html['team1_comp'].str.replace(r'-$','0'))

df_matches_html['team2_comp'] = df_matches_html['team2_comp'].str.replace(r'(/).*','')
df_matches_html['team2_comp'] = pd.to_numeric(df_matches_html['team2_comp'].str.replace(r'-$','0'))

df_matches_html['team1_pass'] = pd.to_numeric(df_matches_html['team1_pass'].str.replace(r'-$','0'))
df_matches_html['team2_pass'] = pd.to_numeric(df_matches_html['team2_pass'].str.replace(r'-$','0'))

df_matches_html['team1_rush'] = pd.to_numeric(df_matches_html['team1_rush'].str.replace(r'-$','0'))
df_matches_html['team2_rush'] = pd.to_numeric(df_matches_html['team2_rush'].str.replace(r'-$','0'))

df_matches_html['team1_block'] = pd.to_numeric(df_matches_html['team1_block'].str.replace(r'-$','0'))
df_matches_html['team2_block'] = pd.to_numeric(df_matches_html['team2_block'].str.replace(r'-$','0'))

df_matches_html['team1_foul'] = pd.to_numeric(df_matches_html['team1_foul'].str.replace(r'-$','0'))
df_matches_html['team2_foul'] = pd.to_numeric(df_matches_html['team2_foul'].str.replace(r'-$','0'))


In [ ]:
df_matches_html.info()

In [ ]:
df_matches_html

## Dataprep: coach rankings

We want to extract the part `CR 149.99` from the scraped coach information field (example `test_string` below). Just as we matches on `Inducements:`, we can match on `CR ` and grab the contents directly after that, stopping when we encounter a whitespace.

We first play around a bit and test until we discover the proper Regular Expression to use :-)


In [ ]:
test_string = 'kingcann Emerging Star CR 149.99 (-0.76)'

pattern = re.compile(r'.*CR (.*)\s\(.*')

match = re.match(pattern, test_string)

if match is not None:
    print(match.group(1)) # group(0) is the whole string
else:
    print("match is none")

test_string

Got 'm! Now that we have figured it out, we can write the code that extracts the coach rankings:

In [ ]:
# PM fix CR being 10x as large


# Dataprep fix match_id
df_matches_html['match_id'] = pd.to_numeric(df_matches_html.match_id) 

# Dataprep: add the coach rankings as separate cols
df_matches_html['coach1_CR'] = df_matches_html['coach1_ranking'].str.extract(r'.*CR (.*)\s\(.*')
df_matches_html['coach2_CR'] = df_matches_html['coach2_ranking'].str.extract(r'.*CR (.*)\s\(.*')

df_matches_html['coach1_CR'] = pd.to_numeric(df_matches_html['coach1_CR'])
df_matches_html['coach2_CR'] = pd.to_numeric(df_matches_html['coach2_CR'])

# abs
df_matches_html['CR_diff'] = np.abs(df_matches_html['coach1_CR'] - df_matches_html['coach2_CR'])
df_matches_html['CR_diff'] = df_matches_html['CR_diff'].astype(float)

# +/-
df_matches_html['cr_diff2'] = df_matches_html['coach1_CR'] - df_matches_html['coach2_CR']

df_matches_html['cr_diff2_bin'] = pd.cut(df_matches_html['cr_diff2'], bins = [-1*float("inf"), -30, -20, -10, -5, 5, 10, 20, 30, float("inf")], 
 labels=['{-Inf,-30]', '[-30,-20]', '[-20,-10]', '[-10,-5]', '[-5,5]', '[5,10]', '[10,20]', '[20,30]', '[30,Inf]']) 

df_matches_html['coach1_CR_bin'] = pd.cut(df_matches_html['coach1_CR'], 
    bins = [0, 135,145, 155, 165, 175, float("inf")], 
    labels=['CR135-', 'CR140', 'CR150', 'CR160','CR170', 'CR175+'])

df_matches_html['coach2_CR_bin'] = pd.cut(df_matches_html['coach2_CR'], 
    bins = [0, 135,145, 155, 165, 175, float("inf")], 
    labels=['CR135-', 'CR140', 'CR150', 'CR160','CR170', 'CR175+'])


In [ ]:
df_matches_html['coach1_CR_bin'].value_counts()

## Dataprep match inducements for each team

The next trick is to use `pandas` `explode()` method (similar to `separate_rows()` in `tidyverse` R) to give each inducement its own row in the dataset.
This creates a dataframe (`inducements`) similar to `df_mbt` with each match generating at least two rows.


In [ ]:
team1_inducements = df_matches_html[['match_id', 'team1_inducements']]
team2_inducements = df_matches_html[['match_id', 'team2_inducements']]

# make column names equal
team1_inducements.columns = team2_inducements.columns = ['match_id', 'inducements']
team1_inducements['team'] = 'team1'
team2_inducements['team'] = 'team2'

# row bind the two dataframes
inducements = pd.concat([team1_inducements, team2_inducements], ignore_index = True)

# convert comma separated string to list
inducements['inducements'] = inducements['inducements'].str.split(',')

# make each element of the list a separate row
inducements = inducements.explode('inducements')

# strip leading and trailing whitespaces
inducements['inducements'] = inducements['inducements'].str.strip()

# create "star player" label
inducements['star_player'] = 0
inducements.loc[inducements['inducements'].str.contains("Star player"), 'star_player'] = 1

# create "card" label
inducements['special_card'] = 0
inducements.loc[inducements['inducements'].str.contains("Card"), 'special_card'] = 1


In [ ]:
inducements

## Add match HTML data to df_matches

Here we add `df_matches_html` to `df_matches`. This contains each players inducements as a single string, not convenient for analysis.




In [ ]:
df_matches = pd.merge(df_matches, df_matches_html, on='match_id', how='left')

## Add specific inducement info to df_matches

The `inducements` dataframe cannot easily be added to `df_matches`. We can however, extract information from `inducements` at the match level and add this to `df_matches`. Here, I show how to add a 1/0 flag `has_sp` that codes for if a match included any star player.

In [ ]:
df_sp = (inducements
            .groupby("match_id")
            .agg(has_sp = ("star_player", "max"))
            .reset_index()
)


df_matches = pd.merge(df_matches, df_sp, on = "match_id", how = "left")

df_matches['match_id'] = pd.to_numeric(df_matches.match_id) 

# steps 4-6 add metdata to matches

The match JSON is the source for division and division name, these are not present in the tournament endpoint. This is a team property.
The tournament JSON Is the source for the group_id (league), these are not present in the match endpoint.
The group JSON is the source for the group name and ruleset.

question: is it possible to add league (number / id), and possibly ruleset,  to the match endpoint?
Not really, because I don't have it stored there
The thing is that teams can be moved across leagues (and therefore rulesets)
So you can't know for sure which was being used at the time of the game
So even if I take league id from the teams endpoint, it might not be correct for all matches played by that team? Correct.
And even for a given league, the base ruleset can change over time (which won't be tracked)
And for a given ruleset, there's no guarantee that the settings are the same over time.
Much in the same way you can't really look at the current state of a team to see which players were in a game for a given match in the past.
You could update nightly for matches really if you wanted to


# Step 4 add tournament info

create list of all tournament ids.
Fetch and store as JSON.
Then process and add tournament names to `df_matches`.

There are some 4K tournaments in the data.

In [ ]:
# make list of all tournaments that need to be fetched

tournament_ids = df_matches['tournament_id'].values

# get unique values by converting to a Python set and back to list
tournament_ids = list(set(tournament_ids))

len(tournament_ids)



In [ ]:
# save tournament API call as gzipped JSON object.
def update_tourney_jsons(tournament_ids, fullrun = 0):
    print("total nr of tournaments in the data:")

    n_tourneys = len(tournament_ids)

    print(n_tourneys)

    if fullrun:
        print('fetching tournament data for ', n_tourneys, ' tourneys')

        for t in range(n_tourneys):
            tournament_id = int(tournament_ids[t])
            if t % 100 == 0: 
                # write progress report
                print(t, end='')
                print(".")

            dirname = "raw/tournament_files/" + str(tournament_id)[0:2]
            if not os.path.exists(dirname):
                os.makedirs(dirname)

            fname_string = dirname + "/tournament_" + str(tournament_id) + ".json.gz"

            # check if file already exists, else scrape it
            try:
                f = open(fname_string, mode = "rb")

            except OSError as e:
                # file not present, scrape it         
                api_string = "https://fumbbl.com/api/tournament/get/" + str(tournament_id)

                response = requests.get(api_string)
                response = response.json()

                with gzip.open(fname_string, mode = "w") as f:
                    f.write(json.dumps(response).encode('utf-8'))  
                    print('x', end = '')
                    f.close()
                time.sleep(0.3)
            else:
                # file already present
                print("o", end = '')
                continue


    else:
        print("full run is off")

In [ ]:
update_tourney_jsons(tournament_ids, fullrun = 1)

## process the tourney json files from disk

In [ ]:
def do_all_the_tourney_stuff(tournament_ids, fullrun):
    
    n_tourneys = len(tournament_ids)

    target = 'raw/df_tourneys_' + time.strftime("%Y%m%d_%H%M%S") + '.h5'

    if fullrun:
        tournament_id = []
        group_id = []
        tournament_type = []
        tournament_progression = []
        tournament_name = []
        tournament_start = []
        tournament_end = []

        print('processing tourney data for ', n_tourneys, ' tournaments')

        for t in range(n_tourneys):    
            tournament_id_tmp = int(tournament_ids[t])
            dirname = "raw/tournament_files/" + str(tournament_id_tmp)[0:2]

            fname_string_gz = dirname + "/tournament_" + str(tournament_id_tmp) + ".json.gz"        
            
            # read compressed json file
            with gzip.open(fname_string_gz, mode = "rb") as f:
                tournament = json.load(f)

            if str(tournament) != 'No such tournament.':
                # grab fields
                tournament_id.append(tournament['id'])
                group_id.append(tournament['group'])
                tournament_type.append(tournament['type'])
                tournament_progression.append(tournament['progression'])
                tournament_name.append(tournament['name'])
                tournament_start.append(tournament['start'])
                tournament_end.append(tournament['end'])   

            if t % 500 == 0: 
                # write tmp data as hdf5 file
                print(t, end='')
                print(".", end='')
        # create list of tuples
        data = zip(tournament_id, group_id, tournament_type, tournament_progression, tournament_name, tournament_start, tournament_end)
        # create dataframe from list
        df_tourneys = pd.DataFrame(data, columns=['tournament_id', 'group_id', 'tournament_type', 'tournament_progression', 
        'tournament_name', 'tournament_start', 'tournament_end'])
        df_tourneys.to_hdf(target, key='df_tourneys', mode='w')   


    else:
        # read from hdf5 file
        df_tourneys = pd.read_hdf('raw/df_tourneys_20230528_143734.h5')

    df_tourneys['tournament_id'] = pd.to_numeric(df_tourneys.tournament_id) 
    df_tourneys['group_id'] = pd.to_numeric(df_tourneys.group_id) 
    df_tourneys['tournament_type'] = pd.to_numeric(df_tourneys.tournament_type) 

    return df_tourneys

df_tourneys = do_all_the_tourney_stuff(tournament_ids, fullrun = 1)

df_tourneys.shape

In [ ]:
df_tourneys.sort_values(by='tournament_id', inplace=True, ascending=False)
df_tourneys = df_tourneys.reset_index(drop=True)



# Step 5 Fetch groups (for groupname and ruleset)

Ok, let me start from the beginning with terminology 🙂
A ruleset is basically the root of the whole thing at this point. This is where rosters and rule options are defined.
A division is an environment for "open play", and used to be the root of the data model before rulesets were a thing.
A league is a special kind of group, which is defined by it having a ruleset configured.
A group is a "meta group" and currently acts mostly as a parent for tournaments
A tournament is the actual scheduling systems put into play.

A "League" is a special kind of "Group"?
\
Yes, they are technically the same, but the league is configured with a custom ruleset (and therefore doesn't use the division default ruleset)
The distinction between a group and a league is very narrow, but has a huge effect.
In a sense the "league" is equivalent to a custom division.
https://discord.com/channels/254387387260469258/739746315449139240/1080863062342504478

In [ ]:
group_ids = df_tourneys['group_id'].values

# get unique values by converting to a Python set and back to list
group_ids = list(set(group_ids))

len(group_ids)

In [ ]:
# save group API call as gzipped JSON object.
def update_group_jsons(group_ids, fullrun = 0):
    print("total nr of groups in the data:")

    n_groups = len(group_ids)

    print(n_groups)

    if fullrun:
        print('fetching group data for ', n_groups, ' groups')

        for t in range(n_groups):
            group_id = int(group_ids[t])
            if t % 100 == 0: 
                # write progress report
                print(t, end='')
                print(".")

            dirname = "raw/group_files/" + str(group_id)[0:2]
            if not os.path.exists(dirname):
                os.makedirs(dirname)

            fname_string = dirname + "/group_" + str(group_id) + ".json.gz"

            # check if file already exists, else scrape it
            try:
                f = open(fname_string, mode = "rb")

            except OSError as e:
                # file not present, scrape it         
                api_string = "https://fumbbl.com/api/group/get/" + str(group_id)

                response = requests.get(api_string)
                response = response.json()

                with gzip.open(fname_string, mode = "w") as f:
                    f.write(json.dumps(response).encode('utf-8'))  
                    print('x', end = '')
                    f.close()
                time.sleep(0.3)
            else:
                # file already present
                print("o", end = '')
                continue


    else:
        print("full run is off")

In [ ]:
update_group_jsons(group_ids, fullrun = 1)

In [ ]:
def process_group_jsons(group_ids, fullrun):

    n_groups = len(group_ids)

    target = 'raw/df_groups_' + time.strftime("%Y%m%d_%H%M%S") + '.h5'

    if fullrun:
        group_id = []
        group_name = []
        ruleset = []

        print('processing group data for ', n_groups, ' groups')

        for t in range(n_groups):    
            group_id_tmp = int(group_ids[t])
            dirname = "raw/group_files/" + str(group_id_tmp)[0:2]

            fname_string_gz = dirname + "/group_" + str(group_id_tmp) + ".json.gz"        
            
            # read compressed json file
            with gzip.open(fname_string_gz, mode = "rb") as f:
                group = json.load(f)

            if str(group) != 'No such group.':
                # grab fields
                group_id.append(group['id'])
                group_name.append(group['name'])
                ruleset.append(group['ruleset'])

            if t % 500 == 0: 
                # write tmp data as hdf5 file
                print(t, end='')
                print(".", end='')

        data = zip(group_id, group_name, ruleset)
        df_groups = pd.DataFrame(data, columns=['group_id', 
        'group_name', 'ruleset'])
        df_groups.to_hdf(target, key='df_groups', mode='w')   


    else:
        # read from hdf5 file
        df_groups = pd.read_hdf('raw/df_groups_20230528_143736.h5')

    df_groups['group_id'] = pd.to_numeric(df_groups.group_id) 
    df_groups['ruleset'] = pd.to_numeric(df_groups.ruleset) 

    return df_groups

df_groups = process_group_jsons(group_ids, fullrun = 1)
df_groups = df_groups.rename(columns={"ruleset": "current_ruleset"})
df_groups.shape

In [ ]:
df_groups

In [ ]:
df_tourneys2 = pd.merge(df_tourneys, df_groups, on = 'group_id', how = 'left') # lost a few

df_tourneys2.shape


## add tourney, group and division data to match data


In [ ]:
df_matches.iloc[[0]]

In [ ]:
df_matches = pd.merge(df_matches, df_tourneys2, on='tournament_id', how='left')


In [ ]:
df_matches.query("match_id == 4421729")
df_matches.shape


# Step 6  Add ruleset_version and division_name to df_matches

Divisions are very important in FUMBBL, every team belongs to a Division.

PM we see that (NAF) matches played previously under ruleset 2228 are now labeled as ruleset 2310? this has a few changes (tier, gold, crossleague)
* Do we also see this in the XML API

Lets have look at the various divisions and leagues, and which rulesets are used.
There are a lot of small leagues being played on FUMBBL, they account for maybe X% of all the matches.

We only look at divisions and leagues with a sufficient volume of matches, or otherwise we do not have sufficient statistics for each race.

So I aggregated the data by division, league and ruleset, and filtered on at least 150 different teams that have played at least once last year.
Apart from the main "Divisions" that are part of FUMBBL, there were a few user-run leagues present in this table, so I looked up their names on FUMBBL and what ruleset is used (BB2016, BB2020 or some other variant). This information (contained in an xlsx) is added to the dataset below.


In [ ]:
res = (df_matches
.groupby([ 'division_id', 'division_name', 'scheduler' , 'year', 'group_id', 'group_name',  'tournament_id' , 'tournament_name'], dropna=False)
.agg(
    n_games = ('match_id', 'count')
)
.reset_index()
.sort_values("n_games", ascending=False)
)

#res.to_excel('group_counts.xlsx')
res

conclusion, major divisions are not found. Those (teams /) matches do not have a tournament id.
But clearly distinguished.


# Save all prepped datasets as HDF5 and CSV files

HD5 files can be read by both Python and [R](https://cran.r-project.org/web/packages/hdf5r/index.html) and preserve column data types.

**Update** the HDF5 schema used by pandas is highly specific to pandas, it is not designed for external use. An R code snippet is available but does not work with table format. For now using the CSVs for R is advised

CSV files are the lingua franca across all data analysis software.

A dataset release consists of three datasets:
* A list of matches, identified by match_id
* A list of matches by team, identified by match_id and team_id
* A list of inducements by match_id

In [ ]:
# use this to locate variables that cannot be serialized by hdf5
#df_matches.loc[:, :'week_date'].info()

In [ ]:
n = 50000  #chunk row size
list_df = [df_matches[i:i+n] for i in range(0,len(df_matches),n)]


In [ ]:
target = 'datasets/current/df_matches'

for i in range(len(list_df)):
    print(list_df[i].shape)
    if i == 0:
        list_df[i].to_hdf(target + '.h5', key='df_matches', mode='w', format = 'table',  complevel = 9, min_itemsize=750)
    else:
        list_df[i].to_hdf(target + '.h5', key='df_matches', append = True, mode='r+', format = 'table', complevel = 9)


In [ ]:
target = 'datasets/current/df_matches'

df_matches.to_csv(target + '.csv')


In [ ]:
target = 'datasets/current/inducements'
inducements.to_hdf(target + '.h5', key='inducements', mode='w', format = 't',  complevel = 9)
inducements.to_csv(target + '.csv')

In [ ]:
target = 'datasets/current/df_tourneys'

df_tourneys.to_hdf(target + '.h5', key='df_tourneys', mode='w', format = 't',  complevel = 9)
df_tourneys.to_csv(target + '.csv')

# Choosing a license for the public dataset

An important part of making data publicly available is being explicit about what is allowed if people want to use the dataset.
However, before we do so, we have to check if **we** are actually allowed to publish the data. This is explained nicely [in a blogpost by Elizabeth Wickes](https://datacarpentry.org/blog/2016/06/data-licensing).

Since our data comes from the **FUMBBL.com** website, we check the [**Privacy policy**](https://fumbbl.com/p/privacy) where all users, including myself have agreed on when signing up. It contains this part which is specific to the unauthenticated API, which we use to fetch the data, as well as additional public match data, such as which inducements are used in a match, and the Coach rankings of the playing coaches that were current when the match was played.

```
Content you provide through the website
All the information you provide through the website is processed by FUMBBL. This includes things such as forum posts, private message posts, blog entries, team and player names and biographies and news comments. Data provided this way is visible by other people on the website and in most cases public even to individuals without accounts (not including private messages), and as such are considered of public interest. If direct personal information is posted in public view, you can contact moderators to resolve this. Match records are also considered content in this context, and is also considered of public interest. This data is collected as the primary purpose of the website and it is of course entirely up to you how much of this is provided to FUMBBL. 

Third party sharing
Some of the public data is available through a public (*i.e. unauthenticated*) API, which shares some of the information provided by FUMBBL users in a way suitable for third-party websites and services to process.

The data available through the unauthenticated API is considered non-personal as it only reflects information that is public by its nature on the website. The authenticated API will only show information connected to the authenticated account.
```

I conclude that since the match data is already considered public content, there is no harm in collecting this public data in a structured dataset and placing this data in a public repository. I also verified this with Christer, the site owner. 


The final step is then to decide what others are allowed to do with this data. In practice, this means choosing a license under which to release the dataset. I decided to choose a CC0 license: this places the data in the public domain, and people can use the dataset as they wish. Citing or mentioning the source of the data would still be appreciated of course.

# List of possible future improvements

* Scraping the players (only most recent version, so no player development history)
* Scraping the rulesets (for example to identify resurrection tournaments where players choose skills and use tiers)

* catch exception: **PM we cannot deal yet with the situation HTTPSConnectionPool(host='fumbbl.com', port=443): Max retries exceeded with url: /api/match/get/4221820 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f4acff12be0>: Failed to establish a new connection: [Errno 110] Connection timed out',))**

* cr_bin variable is gone?
